In [34]:
#Load Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler 
from sklearn.neural_network import MLPClassifier 
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.tree import DecisionTreeClassifier

In [2]:
# #Load Data
data_DB = pd.read_csv("C:/Users/igbin/Downloads/dataset.csv")
data_DB.head()

,hash,millisecond,classification,state,usage_counter,prio,static_prio,normal_prio,policy,vm_pgoff,...,nivcsw,min_flt,maj_flt,fs_excl_counter,lock,utime,stime,gtime,cgtime,signal_nvcsw
0,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914...,0,malware,0,0,3069378560,14274,0,0,0,...,0,0,120,0,3204448256,380690,4,0,0,0
1,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914...,1,malware,0,0,3069378560,14274,0,0,0,...,0,0,120,0,3204448256,380690,4,0,0,0
2,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914...,2,malware,0,0,3069378560,14274,0,0,0,...,0,0,120,0,3204448256,380690,4,0,0,0
3,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914...,3,malware,0,0,3069378560,14274,0,0,0,...,0,0,120,0,3204448256,380690,4,0,0,0
4,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914...,4,malware,0,0,3069378560,14274,0,0,0,...,0,0,120,0,3204448256,380690,4,0,0,0


In [6]:
# #Key Statistics
data_DB.describe()

,millisecond,state,usage_counter,prio,static_prio,normal_prio,policy,vm_pgoff,vm_truncate_count,task_size,...,nivcsw,min_flt,maj_flt,fs_excl_counter,lock,utime,stime,gtime,cgtime,signal_nvcsw
count,100000.000000,1.000000e+05,100000.0,1.000000e+05,100000.000000,100000.0,100000.0,100000.0,100000.000000,100000.0,...,100000.000000,100000.000000,100000.000000,100000.000000,1.000000e+05,100000.000000,100000.000000,100000.00000,100000.0,100000.0
mean,499.500000,1.577683e+05,0.0,3.069706e+09,18183.900070,0.0,0.0,0.0,15312.739510,0.0,...,32.991160,2.053130,117.920240,1.109190,3.204448e+09,385415.451970,4.059310,1.66142,0.0,0.0
std,288.676434,9.361726e+05,0.0,2.963061e+05,4609.792765,0.0,0.0,0.0,3256.475008,0.0,...,52.730176,13.881382,3.116892,2.160466,0.000000e+00,10144.036494,0.822848,3.26304,0.0,0.0
min,0.000000,0.000000e+00,0.0,3.069190e+09,13988.000000,0.0,0.0,0.0,9695.000000,0.0,...,0.000000,0.000000,112.000000,0.000000,3.204448e+09,371782.000000,3.000000,0.00000,0.0,0.0
25%,249.750000,0.000000e+00,0.0,3.069446e+09,14352.000000,0.0,0.0,0.0,12648.000000,0.0,...,1.000000,0.000000,114.000000,0.000000,3.204448e+09,378208.000000,3.000000,0.00000,0.0,0.0
50%,499.500000,0.000000e+00,0.0,3.069698e+09,16159.000000,0.0,0.0,0.0,15245.000000,0.0,...,9.000000,1.000000,120.000000,0.000000,3.204448e+09,383637.000000,4.000000,0.00000,0.0,0.0
75%,749.250000,4.096000e+03,0.0,3.069957e+09,22182.000000,0.0,0.0,0.0,17663.000000,0.0,...,46.000000,1.000000,120.000000,1.000000,3.204448e+09,390324.000000,5.000000,1.00000,0.0,0.0
max,999.000000,4.326605e+07,0.0,3.070222e+09,31855.000000,0.0,0.0,0.0,27157.000000,0.0,...,365.000000,256.000000,120.000000,18.000000,3.204448e+09,421913.000000,7.000000,15.00000,0.0,0.0


In [28]:
# # Create a copy of the dataset
data_DB1 = data_DB.copy()

# # Preprocess data
data_DB1 = pd.get_dummies(data_DB1)
data_DB1 = data_DB1.replace(False, 0)
data_DB1 = data_DB1.replace(True, 1)

# # List columns with only 0 values
zero_columns = data_DB1.columns[(data_DB1 == 0).all()]

# # Drop columns with only 0 values
data_DB1.drop(zero_columns, axis=1, inplace=True)

# # Create x and y variables
x = data_DB1.drop('classification_benign', axis=1).to_numpy()
y = data_DB1['classification_benign'].to_numpy()

# # Split dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, stratify=y, random_state=100)

# # Scale Data
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# # Create and evaluate the model
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=1000, random_state=100)

# # Fit the model
mlp.fit(x_train_scaled, y_train)

# # Predictions
y_pred_train = mlp.predict(x_train_scaled)
y_pred_test = mlp.predict(x_test_scaled)

print('\nModel Evaluation on Training Data:')
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train, y_pred_train))

print('\nModel Evaluation on Testing Data:')
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))


Model Evaluation on Training Data:
[[40000     0]
 [    0 40000]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     40000
           1       1.00      1.00      1.00     40000

    accuracy                           1.00     80000
   macro avg       1.00      1.00      1.00     80000
weighted avg       1.00      1.00      1.00     80000


Model Evaluation on Testing Data:
[[10000     0]
 [    0 10000]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10000
           1       1.00      1.00      1.00     10000

    accuracy                           1.00     20000
   macro avg       1.00      1.00      1.00     20000
weighted avg       1.00      1.00      1.00     20000



### Decision Tree

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier

# # Load Dataset
dataset = pd.read_csv("C:/Users/igbin/Downloads/dataset.csv")
dataset.head()

# # Number of instances and features
num_instances = len(dataset)
num_features = dataset.shape[1]

# # Number of instances from each class (malware/benign)
instances_per_class = dataset['classification'].value_counts()

# # Print the results
print("Number of instances:", num_instances)
print("Number of features:", num_features)
print("Number of instances from each class:")
print(instances_per_class)

# # Create a copy of the dataset
dataset2 = dataset.copy()

# # Preprocess data
dataset2 = pd.get_dummies(dataset2)
dataset2 = dataset2.replace(False, 0)
dataset2 = dataset2.replace(True, 1)

# # List columns with only 0 values
zero_columns = dataset2.columns[(dataset2 == 0).all()]

# # Drop columns with only 0 values
dataset2.drop(zero_columns, axis=1, inplace=True)

# # Create x and y variables
x = dataset2.drop('classification_benign', axis=1).to_numpy()
y = dataset2['classification_benign'].to_numpy()

# # Split dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, stratify=y, random_state=100)

# # Scale Data
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# # Create and evaluate the model
classifier = DecisionTreeClassifier(random_state=100)

# # Cross-validation
cv_scores = cross_val_score(classifier, x_train_scaled, y_train, cv=5)
print("\nCross-Validation Scores:", cv_scores)
print("Mean CV Score:", cv_scores.mean())

# # Fit the model
classifier.fit(x_train_scaled, y_train)

# # Predictions
y_pred_train = classifier.predict(x_train_scaled)
y_pred_test = classifier.predict(x_test_scaled)

# print('\nModel Evaluation on Training Data:')
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train, y_pred_train))

# print('\nModel Evaluation on Testing Data:')
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))


Number of instances: 100000
Number of features: 35
Number of instances from each class:
classification
malware    50000
benign     50000
Name: count, dtype: int64

Cross-Validation Scores: [1. 1. 1. 1. 1.]
Mean CV Score: 1.0

Model Evaluation on Training Data:
[[40000     0]
 [    0 40000]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     40000
           1       1.00      1.00      1.00     40000

    accuracy                           1.00     80000
   macro avg       1.00      1.00      1.00     80000
weighted avg       1.00      1.00      1.00     80000


Model Evaluation on Testing Data:
[[10000     0]
 [    0 10000]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10000
           1       1.00      1.00      1.00     10000

    accuracy                           1.00     20000
   macro avg       1.00      1.00      1.00     20000
weighted avg       1.00      1.00      1.00

### Naive Bayes

In [39]:
#Load Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [40]:
#Load Data
dataset = pd.read_csv("C:/Users/igbin/Downloads/dataset.csv")
dataset.head()

,hash,millisecond,classification,state,usage_counter,prio,static_prio,normal_prio,policy,vm_pgoff,...,nivcsw,min_flt,maj_flt,fs_excl_counter,lock,utime,stime,gtime,cgtime,signal_nvcsw
0,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914...,0,malware,0,0,3069378560,14274,0,0,0,...,0,0,120,0,3204448256,380690,4,0,0,0
1,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914...,1,malware,0,0,3069378560,14274,0,0,0,...,0,0,120,0,3204448256,380690,4,0,0,0
2,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914...,2,malware,0,0,3069378560,14274,0,0,0,...,0,0,120,0,3204448256,380690,4,0,0,0
3,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914...,3,malware,0,0,3069378560,14274,0,0,0,...,0,0,120,0,3204448256,380690,4,0,0,0
4,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914...,4,malware,0,0,3069378560,14274,0,0,0,...,0,0,120,0,3204448256,380690,4,0,0,0


In [41]:
# Create a copy of the dataset
dataset2 = dataset.copy()

In [42]:
# Preprocess data
dataset2 = pd.get_dummies(dataset2)
dataset2 = dataset2.replace(False, 0)
dataset2 = dataset2.replace(True, 1)

In [43]:
# List columns with only 0 values
zero_columns = dataset2.columns[(dataset2 == 0).all()]

In [44]:
# Drop columns with only 0 values
dataset2.drop(zero_columns, axis=1, inplace=True)

In [45]:
# Create x and y variables
x = dataset2.drop('classification_benign', axis=1).to_numpy()
y = dataset2['classification_benign'].to_numpy()

In [46]:
#Create Training and Test Datasets
from sklearn.model_selection import train_test_split
x_train, x_test,y_train, y_test = train_test_split(x, y, stratify=y,test_size=0.2,random_state=100)

#Scale the Data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train2 = sc.fit_transform(x_train)
x_test2 = sc.transform(x_test)

In [47]:
#Script for NB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix  

for name,method in [('Naive Bayes',GaussianNB())]: 
    method.fit(x_train2,y_train)
    predict = method.predict(x_test2)
    target_names=['0','1']
    print('\nEstimator: {}'.format(name)) 
    print(confusion_matrix(y_test,predict))  
    print(classification_report(y_test,predict,target_names=target_names))


Estimator: Naive Bayes
[[10000     0]
 [    0 10000]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10000
           1       1.00      1.00      1.00     10000

    accuracy                           1.00     20000
   macro avg       1.00      1.00      1.00     20000
weighted avg       1.00      1.00      1.00     20000

